*   A principal ideia desse notebook é analisar temperaturas e variáveis climáticas de Juiz de Fora nos últimos dez ou mais. Os dados foram obtidos pela requisição ao INMET.
*   As fases serão: importação, limpeza e depois geração de visualizações
*   Autor: Matheus Valentim







**Importação básica** 

In [3]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
import plotly.express as pl
import datetime

**Montando o drive no PC online disponibilizado pelo google para acessar meus dados que estão lá**

In [4]:
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/jfdados/climatempo'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/jfdados/climatempo


**Criando a base completa de Juiz de Fora**

In [6]:
dic = {"data":str , "hora": str, "ch":float, "cl":float, "cm":float, "nebulosidade":float, "precip":float, "temp_seco":float,
       "temp_umido":float, "temp_orvalho":float,"umidade_rel":float, "veloc_vento":float, "erro":str}

In [7]:
df_completo = pd.read_csv("tempojf.csv",sep = ';', skiprows=10, dtype= dic, decimal='.')

In [8]:
df_completo.columns = ["data", "hora", "ch", "cl", "cm", "nebulosidade", "precip", "temp_seco", "temp_umido", "temp_orvalho",
                       "umidade_rel", "veloc_vento", "erro"]
df_2 = df_completo[["data", "hora", "ch", "cl", "cm", "nebulosidade", "precip", "temp_seco", "temp_umido", "temp_orvalho",
                       "umidade_rel", "veloc_vento"]]

#df_2['temp_seco'] = df_2['temp_seco'].str.replace(",", ".").astype('float64')
df_2['hora'] = df_2['hora'].astype(str)
#df_2['precip'] = df_2['precip'].str.replace(",", ".").astype('float64')
#df_2['nebulosidade'] = df_2['nebulosidade'].str.replace(",", ".").astype('float64')

def ajuste(variavel):
  df_2[variavel] = df_2[variavel].str.replace(",", ".").astype('float64')

for i in ['temp_seco', 'precip', 'nebulosidade']:
  ajuste(i)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
figura = pl.scatter(df_2, x="data", y="temp_seco", color="hora")
                 #size='petal_length', hover_data=['petal_width'])
figura.show()

**Aglomerando por dia para médias diárias**

In [10]:
df_3 = df_2.groupby("data", as_index=False)[["temp_seco", "precip", 'nebulosidade']].agg(media_diaria = ('temp_seco','mean'),
     precipitacao_diaria= ('precip','mean'), nebulosidade_media = ('nebulosidade', 'mean'))

#df.groupby(['col1', 'col2'])[['col3', 'col4']].sum()


In [32]:
figura_agrupada = pl.scatter(df_3, x="data", y='media_diaria',  color = 'nebulosidade_media', 
                                                title = "Médias diárias de temperatura e nebulosidade em Juiz de Fora",
                             labels={
                     "media_diaria": "Temperatura Média (Celsius)",
                     "nebulosidade_media": "Nebulosidade Média"
                 },
                             color_continuous_scale=pl.colors.sequential.Plasma_r)
figura_agrupada.show()

**estatísticazinhas básicas para temp, precipitacao e nebulosidade por ano - nenhuma grande mudança**

In [48]:
df_3['ano'] = df_3['data'].str.slice(0,4)
df_ano = df_3.groupby('ano', as_index=False)[['media_diaria', 'precipitacao_diaria', 'nebulosidade_media']].agg(np.mean)
df_recente = df_3[df_3['ano'].isin(['2019','2020'])]

In [49]:
pl.histogram(df_recente, x= 'media_diaria', color='ano', barmode='overlay')



---



0.   heatmap do calor mesmo 
1.   quando chove faz... como as temperaturas sao mais centralizadas ao em torno da média quando chove
2.   e... comparando diferentes grupos: diferentes julhos
3.   varias distribuicoes em bins (histograma) de diferentes meses, verão, inverno para várias variáveis
4. descritiva de tempo em juiz de fora: variancia de temperatura nas estaçoes e nos meses
5. variancia de temperaturas no mes do inverno ta aumentando? - perda de estabilidade



